In [ ]:
import cv2
import os
import numpy as np


In [ ]:

def composite_images(background_path, foreground_path, output_path):
    """
    두 이미지를 합성합니다. 전경 이미지에 알파 채널이 있는 경우 투명도를 고려합니다.

    Args:
        background_path (str): 배경 이미지 파일 경로.
        foreground_path (str): 전경 이미지 파일 경로 (알파 채널 포함 가능).
        output_path (str): 합성된 이미지를 저장할 파일 경로.
    """
    # 배경 이미지 로드
    background = cv2.imread(background_path)
    if background is None:
        print(f"오류: 배경 이미지 '{background_path}'를 로드할 수 없습니다.")
        return

    # 전경 이미지 로드 (IMREAD_UNCHANGED를 사용하여 알파 채널도 로드)
    foreground = cv2.imread(foreground_path, cv2.IMREAD_UNCHANGED)
    if foreground is None:
        print(f"오류: 전경 이미지 '{foreground_path}'를 로드할 수 없습니다.")
        return

    # 전경 이미지의 크기 및 위치 설정
    bg_height, bg_width, _ = background.shape
    fg_height, fg_width, _ = foreground.shape

    # 바늘 이미지를 배경 이미지의 중앙에 대략적으로 배치
    offset_x = (bg_width - fg_width) // 2
    offset_y = (bg_height - fg_height) // 2

    # 전경 이미지의 ROI (Region of Interest) 설정
    y1, y2 = max(0, offset_y), min(bg_height, offset_y + fg_height)
    x1, x2 = max(0, offset_x), min(bg_width, offset_x + fg_width)

    # 배경 이미지의 해당 ROI 추출
    roi = background[y1:y2, x1:x2]

    # 전경 이미지의 알파 채널이 있는지 확인
    if foreground.shape[2] == 4:  # RGBA 이미지인 경우
        # OpenCV의 cvtColor 함수와 호환되도록 데이터 타입을 float32로 변환합니다.
        alpha_channel = (foreground[:, :, 3] / 255.0).astype(np.float32)
        # RGB 채널 분리
        rgb_channels = foreground[:, :, :3]

        # 마스크 생성 (알파 채널을 3채널로 확장)
        alpha_factor = cv2.cvtColor(alpha_channel, cv2.COLOR_GRAY2BGR)

        # 배경 ROI와 전경 이미지 합성
        # 공식: output = foreground * alpha + background * (1 - alpha)
        composed_roi = rgb_channels * alpha_factor + roi * (1 - alpha_factor)

        # 배경 이미지에 합성된 ROI를 다시 삽입
        background[y1:y2, x1:x2] = composed_roi

    else:  # RGB 이미지인 경우 (투명도 없음)
        # 단순히 덮어씌웁니다.
        background[y1:y2, x1:x2] = foreground

    # 결과 이미지 저장
    cv2.imwrite(output_path, background)
    print(f"합성된 이미지가 '{output_path}'에 저장되었습니다.")


In [ ]:

# 파일 경로 설정
image_folder = "image/pressure_gauge"
background_image_path = os.path.join(image_folder, "pressure_gauge.png")
foreground_image_path = os.path.join(image_folder, "niddle.png")
output_image_path = "composited_gauge.png"

# 이미지 합성 실행
composite_images(background_image_path, foreground_image_path, output_image_path)

합성된 이미지가 'composited_gauge.png'에 저장되었습니다.
